# Example: Sorted Allocation of Balance

## Introduction

This notebook demonstrates how to allocate an existing balance across multiple financial obligations in a sinking fund system.

### Why Allocate Existing Balances?

In real-world scenarios, you'll often start with an existing balance in your sinking fund rather than from zero. This happens in several common situations:

1. **Transitioning from an unstructured to structured approach**: You may have been saving for future expenses without a formal system, and now want to organize those funds into specific envelopes.

2. **Windfall allocation**: Perhaps you received a tax refund, bonus, or gift that you want to distribute across multiple future expenses.

3. **Account consolidation**: You might be combining multiple savings accounts into a single sinking fund system.

4. **Rebalancing existing allocations**: As financial priorities shift, you may need to reallocate your existing sinking fund balance to better align with your current needs.

5. **Starting mid-way through a financial year**: Many expenses (like property taxes, insurance premiums) operate on annual cycles, and you may be implementing your sinking fund system in the middle of these cycles with partially saved amounts.

### The Allocation Challenge

When you have a lump sum to allocate across multiple future expenses, you need a systematic approach to determine which expenses should receive funding priority. The allocation strategies demonstrated in this notebook address this challenge by providing different methods to prioritize your expenses.

Each strategy offers different advantages:

- **Cascade (due-date priority)**: Ensures your most imminent expenses are fully funded first
- **Debt Snowball (smallest-first)**: Gives the psychological win of completely funding smaller expenses 
- **Custom priority**: Allows you to implement your own allocation logic based on personal priorities

Let's explore how these allocation strategies work with a real-world example.

## Imports

In [1]:
########################################################################
## FOR NOTEBOOKS ONLY: ADD THE PROJECT ROOT TO THE PYTHON PATH
########################################################################

import os
import sys

sys.path.insert(
    0, os.path.abspath(os.path.join(os.getcwd(), '..'))
)

In [2]:
import datetime

from sinkingfund.allocation.sorted import SortedAllocator
from sinkingfund.models.bills import Bill
from sinkingfund.models.envelope import Envelope

## Create Envelopes for Bills

In this example, lets create envelopes for three recurring bills (they don't have to be recurring, but it's a good example). We plan to contribute to these enveolves bi-weekly, so every 14 days.

Car Insurnace:  
* First due date (start date): 2026-04-24
* Amount due: $800
* Contribution interval: 14 days
* Recurring: True
* Frequency: Semi-annual

Car Registration:  
* First due date (start date): 2026-05-01
* Amount due: $300
* Contribution interval: 14 days
* Recurring: True
* Frequency: Annual

Property Taxes:  
* First due date (start date): 2026-11-01
* Amount due: $5,000
* Contribution interval: 14 days
* Recurring: True
* Frequency: Annual

### Define the Bills

In [3]:
bills = [
    {
        'bill_id': 'car_insurance',
        'service': 'Insurance',
        'amount_due': 800,
        'recurring': True,
        'start_date': datetime.date(2026, 4, 24),
        'frequency': 'monthly',
        'interval': 6
    },
    {
        'bill_id': 'car_registration',
        'service': 'Registration',
        'amount_due': 300,
        'recurring': True,
        'start_date': datetime.date(2026, 5, 1),
        'frequency': 'annual',
    },
    {
        'bill_id': 'property_taxes',
        'service': 'Taxes',
        'amount_due': 5000,
        'recurring': True,
        'start_date': datetime.date(2026, 11, 1),
        'frequency': 'annual',
        'interval': 1
    }
]

bills = [Bill(**b) for b in bills]

print(bills[0])

Bill(bill_id='car_insurance', service='Insurance', amount_due=800, recurring=True, due_date=None, start_date=datetime.date(2026, 4, 24), end_date=None, frequency='monthly', interval=6)


### Create Envelopes

We will use an interval of 14 days for the contribution schedule. This suggests that you get paid every 14 days, as you would likely want to line up your sinking fund contributions with your pay schedule.

In [4]:
envelopes = [Envelope(bill=b, interval=14) for b in bills]

print(envelopes[0])

Envelope(bill=Bill(bill_id='car_insurance', service='Insurance', amount_due=800, recurring=True, due_date=None, start_date=datetime.date(2026, 4, 24), end_date=None, frequency='monthly', interval=6), remaining=None, allocated=None, interval=14, schedule=None)


## Allocate Existing Balance

Say you have a current balance of $1,000 on 2026-01-01. Since this is a sinking fund, let's assume any balance in the account is dedicated to paying off the bills. Simply change the balance if you want to change that assumption.

This is the basis for choosing an allocation strategy: We need to select a method for allocating the existing balance to the bills. In this notebook, we will explore a *sorted* allocation strategy, where we choose a feature of our bills on which we sort the bills, and then allocate the balance to the bills in that order. The `SortedAllocator` class is used to allocate the balance to the bills in a sorted order and generally it accepts any callable that returns a sort key for a bill.

### Starting Balance

In [5]:
start_date = datetime.date(2026, 1, 1)
balance = 1000

### Cascading Allocation

Cascading allocation or "waterfall" allocation ensures your most imminent expenses are fully funded first. This is probably the most common allocation strategy.

In [6]:
allocator = SortedAllocator(sort_key='cascade')
allocator.allocate(envelopes=envelopes, balance=balance, curr_date=start_date)

Let's see what happened. First, verify that the total amount allocated to the bills equals the starting balance. As we can see, `True` is returned, meaning the total allocated is equal to the starting balance.

In [7]:
print(sum([e.allocated for e in envelopes if e.allocated is not None]) == balance)

True


Now, we can look at that `allocated` and `remaining` amounts for each envelope. Since the car insurance is due in 2 months, it received the bulk of the allocation and is fully funded. The car registration, which is due in 5 months, received what was left after funding the car insurance, and property taxes, which are due in 8 months, received none.

In [8]:
for e in envelopes:
    print(
    f"""
    Bill: {e.bill.bill_id}
    Amount Due: {e.bill.amount_due}
    Allocated: {e.allocated}
    Remaining: {e.remaining}
    """
   )


    Bill: car_insurance
    Amount Due: 800
    Allocated: 800
    Remaining: 0
    

    Bill: car_registration
    Amount Due: 300
    Allocated: 200
    Remaining: 100
    

    Bill: property_taxes
    Amount Due: 5000
    Allocated: 0
    Remaining: 5000
    


### Debt Snowball Allocation

Debt Snowball allocation is a strategy that gives the psychological win of completely funding smaller expenses first. It sorts the bills by amount due, and allocates the balance to the smallest bills first.

In [9]:
allocator = SortedAllocator(sort_key='debt_snowball')
allocator.allocate(envelopes=envelopes, balance=balance, curr_date=start_date)

Verify that the total amount allocated to the bills equals the starting balance.

In [10]:
print(sum([e.allocated for e in envelopes if e.allocated is not None]) == balance)

True


Now, let's see the `allocated` and `remaining` amounts for each envelope. In this case, the car registration, while due in 5 months, is the smallest bill amount, so it received the bulk of the allocation and is fully funded. The car insurance, which is due in 2 months, received what was left after funding the car registration, and property taxes, which are due in 8 months, received none.

In [11]:
for e in envelopes:
    print(
    f"""
    Bill: {e.bill.bill_id}
    Amount Due: {e.bill.amount_due}
    Allocated: {e.allocated}
    Remaining: {e.remaining}
    """
   )


    Bill: car_insurance
    Amount Due: 800
    Allocated: 700
    Remaining: 100
    

    Bill: car_registration
    Amount Due: 300
    Allocated: 300
    Remaining: 0
    

    Bill: property_taxes
    Amount Due: 5000
    Allocated: 0
    Remaining: 5000
    


### Custom Sorting

Below we show an advanced example of a custom sorting strategy. We sort the bills by the priority of the service type. The structure of this function is defined by the `SortKey` protocol in the `sorted.py` file. As long as the function adheres to the protocol, it will work with the `SortedAllocator` class.

In this case, we define a dictionary of priorities for the bill IDs, effecitvely overriding the default sorting and manually sorting the bills by the priority. Not all bills needs to be provided as the default priority is set if a bill is not found in the priorities dictionary.

You might be wondering what are the order of the priorities? High -> low, or low-high? It really doesn't matter as the sorting can be done in either ascending or descending order. The `SortedAllocator` will reverse the order of the bills if the `reverse` flag is set to `True`. So, the choice of order is really up to you.

In [14]:
def priority_sort(bill: 'BillInstance', priorities: dict[str, int]) -> int:
    """
    Sort the bills by the priority of the service type.

    Parameters
    ----------
    bill: BillInstance
        The bill to sort.
    priorities: dict[str, int]
        The priorities of the service types.

    Returns
    -------
    int
        The priority of the service type.
    """

    # Iterate over the priorities and return the priority of the bill.
    for service, priority in priorities.items():

        # If the service type is in the bill, return the priority.
        if bill.bill_id.lower() == service.lower():
            return priority

    # Default low priority.
    return 99

In [15]:
# Initialize the allocator with the priority sort function and allocate
# the balance to the envelopes.
allocator = SortedAllocator(sort_key=priority_sort)

allocator.allocate(
    envelopes=envelopes, balance=balance, curr_date=start_date,
    priorities={'car_insurance': 1, 'car_registration': 2, 'property_taxes': 3}
)

Verify that the total amount allocated to the bills equals the starting balance.

In [16]:
print(sum([e.allocated for e in envelopes if e.allocated is not None]) == balance)

True


In this case, the car insurance is funded first, then car registration, then property taxes.

In [17]:
for e in envelopes:
    print(
    f"""
    Bill: {e.bill.bill_id}
    Amount Due: {e.bill.amount_due}
    Allocated: {e.allocated}
    Remaining: {e.remaining}
    """
   )


    Bill: car_insurance
    Amount Due: 800
    Allocated: 800
    Remaining: 0
    

    Bill: car_registration
    Amount Due: 300
    Allocated: 200
    Remaining: 100
    

    Bill: property_taxes
    Amount Due: 5000
    Allocated: 0
    Remaining: 5000
    


However, let's reverse the order of the priorities and see what happens. Now, property taxes is funded first, then car registration, then car insurance. Given the current balance, the outcome is that property taxes is funded partially with the entire balance, and the remaining bills receive no allocation.

In [18]:
# Initialize the allocator with the priority sort function and allocate
# the balance to the envelopes.
allocator = SortedAllocator(sort_key=priority_sort, reverse=True)

allocator.allocate(
    envelopes=envelopes, balance=balance, curr_date=start_date,
    priorities={'car_insurance': 1, 'car_registration': 2, 'property_taxes': 3}
)

for e in envelopes:
    print(
    f"""
    Bill: {e.bill.bill_id}
    Amount Due: {e.bill.amount_due}
    Allocated: {e.allocated}
    Remaining: {e.remaining}
    """
   )


    Bill: car_insurance
    Amount Due: 800
    Allocated: 0
    Remaining: 800
    

    Bill: car_registration
    Amount Due: 300
    Allocated: 0
    Remaining: 300
    

    Bill: property_taxes
    Amount Due: 5000
    Allocated: 1000
    Remaining: 4000
    
